In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
import pickle
import dask.dataframe as dd
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import subprocess
import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.stats import spearmanr
import numpy as np
import cvxpy as cp

/home/tbellagio/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
!pwd

/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv


In [3]:
## bring the founder 
af_founder = pd.read_csv('../leave_1_out/af_founder.csv')

In [4]:
delta_p_normed_file = 'delta_p_normed.csv'
delta_p_normed = dd.read_csv(delta_p_normed_file, sep = ',')

In [5]:
delta_p_normed = delta_p_normed.compute() 

In [6]:
delta_p_normed = delta_p_normed.reset_index(drop=True)

In [7]:
genotype_counts = pd.read_csv('../leave_1_out/genotype_counts_fullgenome231ecotypes.csv').drop('Unnamed: 0', axis=1)

In [8]:
path_ecotypes = '/carnegie/nobackup/scratch/xwu/grenet/merged_frequency/merged_ecotype_frequency.txt'
real_ef = pd.read_csv(path_ecotypes, sep = '\t')

In [9]:
with open('splits_samples.pkl', 'rb') as file:
    splits_samples = pickle.load(file)

len(splits_samples) ## 100

100

In [10]:
def find_files(directory, filename):
    """
    Search for files with a specific name within all subdirectories of a given directory.

    Parameters:
        directory (str): The root directory from which to start the search.
        filename (str): The name of the file to search for.

    Returns:
        list: A list of full paths to the files that match the given filename.
    """
    matches = []  # List to store the full paths of matching files
    for root, dirs, files in os.walk(directory):
        if filename in files:
            full_path = os.path.join(root, filename)
            matches.append(full_path)
    return matches

# Usage example
root_directory = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/'  # Update this to the directory you want to search
target_filename = 'predicted_allele_freq_bio1.csv'
predicted_delta_p_normed_bio1_files = find_files(root_directory, target_filename)

In [11]:
# Usage example
root_directory = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/'  # Update this to the directory you want to search
target_filename = 'snps_indices_bio1.csv'
snps_indices_bio1_files = find_files(root_directory, target_filename)

In [12]:
split_number = 10

In [20]:
snps_indices_bio1_file

'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_8/snps_indices_bio1.csv'

In [25]:
for split_number in range(100):
    predicted_delta_p_normed_bio1_file = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_{split_number}/predicted_allele_freq_bio1.csv'
    snps_indices_bio1_file = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_{split_number}/snps_indices_bio1.csv'
    clumping_file = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_{split_number}/output_clumping.clumped'
    dict_rpos_index0_file = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_{split_number}/dict_rpos_index0.pkl'

    output_file_af = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_{split_number}/r2_pred_allele_freq.csv' 
    output_file_ef = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_{split_number}/r2_pred_ecotype_freq.csv' 
    output_file_sp = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_{split_number}/spcorr_pred_ecotype_freq.csv' 
    
    #split1 
    train_samples = splits_samples[split_number][0]
    test_samples = splits_samples[split_number][1]
    
    predicted_delta_p_normed_bio1 = pd.read_csv(predicted_delta_p_normed_bio1_file).drop('Unnamed: 0',axis=1)

    snps_indices_bio1_old = pd.read_csv(snps_indices_bio1_file)


    if len(snps_indices_bio1_old) == 0:
        print(f'No snps split {split_number}')
    else:  
        
        ## load the dictionary with the index0 and real snp id
        with open(dict_rpos_index0_file, 'rb') as file:
            dict_rpos_index0 = pickle.load(file)
    
        # load the result of clumping
        clumping = pd.read_csv(clumping_file, delim_whitespace=True)
        # use the dictionary to get a column with the index of the clumped snps 
        clumping['index0'] = clumping['SNP'].replace(dict_rpos_index0)
        
        snps_0index = clumping['index0']
        snps_0index = snps_0index.to_list()
        
        genotype_counts_filt = genotype_counts.iloc[snps_0index, :]
        
        ## retrieving the old index to filter the allele freq prediciton
        snps_1index = np.array(snps_0index) + 1
        
        snps_indices_bio1_old = snps_indices_bio1_old[snps_indices_bio1_old['x'].isin(snps_1index)]
        
        predicted_delta_p_normed_bio1.columns = predicted_delta_p_normed_bio1.columns.str.replace('V','').astype(int)
        
        snps_indices_bio1_old.columns = ['old_index1short', 'snps_indices_bio1_old_full']
        
        predicted_delta_p_normed_bio1 = predicted_delta_p_normed_bio1[snps_indices_bio1_old['old_index1short']]
            
        ## delta_p_normed train 
        delta_p_normed_train = delta_p_normed[train_samples]
        #delta_p_normed_mean = delta_p_normed_train.mean(axis=1)
        #mean_af_sel = delta_p_normed_mean.iloc[snps_0index,]
        
        ## allele freq test
        delta_p_normed_test = delta_p_normed[test_samples]
        
        delta_p_normed_sel = delta_p_normed_test.iloc[snps_0index,:]
        
        delta_p_normed_sel = delta_p_normed_sel.sort_index()
        
        ## subract the mean to see what i get 
        #predicted_delta_p_normed_bio1 = predicted_delta_p_normed_bio1.sub(mean_af_sel.values,axis=1)
        
        predicted_delta_p_normed_bio1t = predicted_delta_p_normed_bio1.T
        
        predicted_delta_p_normed_bio1t.columns = delta_p_normed_sel.columns
        
        r2_af = {}
        for i in delta_p_normed_sel.columns:
            # Reshape the data to fit the model
            X = delta_p_normed_sel[i].values.reshape(-1, 1)  # Independent variable
            y = predicted_delta_p_normed_bio1t[i].values # Dependent variable
        
            # Fit the linear model
            model = LinearRegression().fit(X, y)
        
            # Predict the values
            y_pred = model.predict(X)
        
            # Calculate R^2
            r_squared = r2_score(y, y_pred)
            r2_af[i] = r_squared
        
        r2_af = pd.DataFrame(list(r2_af.items()), columns=['Index', 'Value'])
        r2_af.columns = ['sample', 'r2'] 
        
        r2_af.to_csv(output_file_af)
        
        
        af_founder_sel = af_founder.iloc[snps_0index]
        
        af_founder_sel = np.array(af_founder_sel['af'])
        
        predicted_allele_freq_bio1 = (predicted_delta_p_normed_bio1.multiply(af_founder_sel, axis=1)) + af_founder_sel
        
        genotype_counts_array = genotype_counts_filt.to_numpy()
        
        pred_ecotypes_frequency = {}
        for sample in range(predicted_allele_freq_bio1.shape[0]):
            ## just select the frist sample
            one_sample = predicted_allele_freq_bio1.iloc[sample,:]
            
            y = np.asarray(one_sample).T * 2
            
            h = cp.Variable(231)
            
            product = np.asarray(genotype_counts_array) @ h 
            
            diff = product - y
            
            constraints = [0 <= h, sum(h) == 1]
            
            problem = cp.Problem(cp.Minimize(cp.norm(diff)), constraints)
            
            # Solve the optimization problem using the SCS solver, with verbose output disabled.
            problem.solve(solver='SCS', verbose=False,eps=1e-6)
            
            pred_ecotypes_frequency[sample] = h.value
        
        
        ## get the real ecotype freq 
        
        real_ef_test = real_ef[test_samples]
        
        pred_ecotypes_frequency = pd.DataFrame(pred_ecotypes_frequency)
        
        pred_ecotypes_frequency.columns = real_ef_test.columns
        
        r2_ef = {}
        spcorrelation_ef = {}
        
        for i in real_ef_test.columns:
        
            # Get the ranks of the data
            X_ranked = real_ef_test[i].sort_values().reset_index()['index'] #.reshape(-1, 1)  # Independent variable
            y_ranked = pred_ecotypes_frequency[i].sort_values().reset_index()['index']  # Dependent variable
        
            # Compute Spearman's rank correlation
            correlation, _ = spearmanr(X_ranked, y_ranked)
            spcorrelation_ef[i] = correlation
        
            # Reshape the data to fit the model
            X = real_ef_test[i].values.reshape(-1, 1)  # Independent variable
            y = pred_ecotypes_frequency[i].values  # Dependent variable
        
            # Fit the linear model
            model = LinearRegression().fit(X, y)
        
            # Predict the values
            y_pred = model.predict(X)
        
            # Calculate R^2
            r_squared = r2_score(y, y_pred)
            r2_ef[i] = r_squared
        
        
        spcorrelation_ef = pd.DataFrame(list(spcorrelation_ef.items()), columns=['Index', 'Value'])
        spcorrelation_ef.columns = ['sample', 'r2'] 
        
        r2_ef = pd.DataFrame(list(r2_ef.items()), columns=['Index', 'Value'])
        r2_ef.columns = ['sample', 'r2'] 
        
        spcorrelation_ef.to_csv(output_file_sp)
        r2_ef.to_csv(output_file_ef)
        
        print(f'done{split_number}')

done0
done1
done2
done3
done4
done5
done6
done7
No snps split 8
done9
done10
done11
done12
done13
done14
done15
done16
done17
done18
done19
done20
done21
done22
done23
done24
done25
done26
done27
done28
done29
done30
done31
done32
done33
done34
done35
done36
done37
done38
done39
done40
done41
done42
done43
done44
done45
done46
done47
done48
done49
done50
done51
done52
done53
done54
done55
done56
done57
done58
done59
done60
done61
done62
done63
done64
done65
done66
done67
done68
done69
done70
done71
done72
done73
done74
done75
done76
done77
done78
done79
done80
done81
done82
done83
done84
done85
done86
done87
done88
done89
done90
done91
done92
done93
done94
done95
done96
done97
done98
done99


In [12]:
for split_number in range(100):
    predicted_allele_freq_bio1_file = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_{split_number}/predicted_allele_freq_bio1.csv'
    snps_indices_bio1_file = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_{split_number}/snps_indices_bio1.csv'
    output_file_ef = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_{split_number}/r2_pred_ecotype_freq_wmean.csv' 
    output_file_af = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_{split_number}/r2_pred_allele_freq_wmean.csv' 

    
    #split1 
    train_samples = splits_samples[split_number][0]
    test_samples = splits_samples[split_number][1]
    
    predicted_allele_freq_bio1 = pd.read_csv(predicted_allele_freq_bio1_file).drop('Unnamed: 0',axis=1)
    
    snps_indices_bio1 = pd.read_csv(snps_indices_bio1_file)
    print(predicted_allele_freq_bio1.head())    
    snps_0index = snps_indices_bio1['x'] - 1 
    
    snps_0index = snps_0index.to_list()
    
    genotype_counts_filt = genotype_counts.iloc[snps_0index, :]
    print(genotype_counts_filt.head())
    genotype_counts_array = genotype_counts_filt.to_numpy()
    
    pred_ecotypes_frequency = {}
    for sample in range(predicted_allele_freq_bio1.shape[0]):
        ## just select the frist sample
        one_sample = predicted_allele_freq_bio1.iloc[sample,:]
        
        y = np.asarray(one_sample).T * 2
        
        h = cp.Variable(231)
    
        genotype_counts_array = genotype_counts_filt.to_numpy()
        
        product = np.asarray(genotype_counts_array) @ h 
        
        diff = product - y
        
        constraints = [0 <= h, sum(h) == 1]
        
        problem = cp.Problem(cp.Minimize(cp.norm(diff)), constraints)
        
        # Solve the optimization problem using the SCS solver, with verbose output disabled.
        problem.solve(solver='SCS', verbose=False,eps=1e-6)
        
        pred_ecotypes_frequency[sample] = h.value
    
    
    ## get the real ecotype freq 
    
    real_ef_test = real_ef[test_samples]
    
    pred_ecotypes_frequency = pd.DataFrame(pred_ecotypes_frequency)
    
    pred_ecotypes_frequency.columns = real_ef_test.columns
    
    r2_ef = {}
    
    for i in real_ef_test.columns:
        # Reshape the data to fit the model
        X = real_ef_test[i].values.reshape(-1, 1)  # Independent variable
        y = pred_ecotypes_frequency[i].values  # Dependent variable
    
        # Fit the linear model
        model = LinearRegression().fit(X, y)
    
        # Predict the values
        y_pred = model.predict(X)
    
        # Calculate R^2
        r_squared = r2_score(y, y_pred)
        r2_ef[i] = r_squared
    
    r2_ef = pd.DataFrame(list(r2_ef.items()), columns=['Index', 'Value'])
    r2_ef.columns = ['sample', 'r2'] 
    
    r2_ef.to_csv(output_file_ef)
    
    ## allele_freq train 
    allele_freq_train = allele_freq[train_samples]
    allele_freq_mean = allele_freq_train.mean(axis=1)
    #mean_af_sel = allele_freq_mean.iloc[snps_0index,]

    ## allele freq test
    allele_freq_test = allele_freq[test_samples]

    allele_freq_sel = allele_freq_test.iloc[snps_0index,:]
    
    allele_freq_sel = allele_freq_sel.sort_index()
    
    ## subract the mean to see what i get 
    #predicted_allele_freq_bio1 = predicted_allele_freq_bio1.sub(mean_af_sel.values,axis=1)
    
    predicted_allele_freq_bio1t = predicted_allele_freq_bio1.T
    
    predicted_allele_freq_bio1t.columns = allele_freq_sel.columns
    
    r2_af = {}
    for i in allele_freq_sel.columns:
        # Reshape the data to fit the model
        X = allele_freq_sel[i].values.reshape(-1, 1)  # Independent variable
        y = predicted_allele_freq_bio1t[i].values # Dependent variable
    
        # Fit the linear model
        model = LinearRegression().fit(X, y)
    
        # Predict the values
        y_pred = model.predict(X)
    
        # Calculate R^2
        r_squared = r2_score(y, y_pred)
        r2_af[i] = r_squared
    
    r2_af = pd.DataFrame(list(r2_af.items()), columns=['Index', 'Value'])
    r2_af.columns = ['sample', 'r2'] 
    
    r2_af.to_csv(output_file_af)
    print(f'done{split_number}')

        V1        V2        V3        V4        V5        V6        V7  \
0  0.06982  0.055864  0.049478  0.041729  0.585285  0.024016  0.003325   
1  0.06982  0.055864  0.049478  0.041729  0.585285  0.024016  0.003325   
2  0.06982  0.055864  0.049478  0.041729  0.585285  0.024016  0.003325   
3  0.06982  0.055864  0.049478  0.041729  0.585285  0.024016  0.003325   
4  0.06982  0.055864  0.049478  0.041729  0.585285  0.024016  0.003325   

         V8        V9       V10  ...      V673      V674      V675      V676  \
0  0.014479  0.656701  0.638682  ...  0.144132  0.524631  0.524631  0.524631   
1  0.014479  0.656701  0.638682  ...  0.144132  0.524631  0.524631  0.524631   
2  0.014479  0.656701  0.638682  ...  0.144132  0.524631  0.524631  0.524631   
3  0.014479  0.656701  0.638682  ...  0.144132  0.524631  0.524631  0.524631   
4  0.014479  0.656701  0.638682  ...  0.144132  0.524631  0.524631  0.524631   

       V677      V678      V679      V680      V681      V682  
0  0.17269

KeyboardInterrupt: 